In [18]:
pip install -Uq upgini catboost

Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd

In [20]:
df = pd.read_csv(r"C:\Users\melwi\Desktop\ML\project1\train.csv")
df = df.sample(n = 12000, random_state=0)

In [21]:
df.head(20)

,date,store,item,sales
335813,2017-07-14,4,19,56
630838,2015-05-19,6,35,45
365685,2014-05-01,1,21,48
322781,2016-11-06,7,18,85
151590,2013-02-02,4,9,46
572011,2014-04-19,4,32,39
786135,2015-08-14,1,44,34
611462,2017-04-28,5,34,21
563327,2015-07-09,9,31,76
820691,2015-03-29,10,45,98


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 335813 to 811189
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    12000 non-null  object
 1   store   12000 non-null  int64 
 2   item    12000 non-null  int64 
 3   sales   12000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 468.8+ KB


In [23]:
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df['date'] = pd.to_datetime(df['date'])

df.sort_values("date", inplace = True)
df.reset_index(inplace = True, drop = True)


In [24]:
df.head()

,date,store,item,sales
0,2013-01-01,2,3,19
1,2013-01-01,7,5,5
2,2013-01-01,10,21,33
3,2013-01-01,3,27,11
4,2013-01-01,4,9,19


In [25]:
train = df[df['date'] < "2017-01-01"]
test = df[df['date'] >= "2017-01-01"]

In [26]:
train_features = train.drop(columns = ['sales'])
train_target = train['sales']

In [27]:
test_features = test.drop(columns = ['sales'])
test_target = test['sales']

In [28]:
test_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2422 entries, 9578 to 11999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2422 non-null   datetime64[ns]
 1   store   2422 non-null   object        
 2   item    2422 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 75.7+ KB


In [29]:
train_target

0       19
1        5
2       33
3       11
4       19
        ..
9573    43
9574    78
9575    21
9576    52
9577    17
Name: sales, Length: 9578, dtype: int64

Enrich feature using upgini

In [30]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features, train_target, 
             eval_set = [(test_features, test_target)])

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample




Column name,Status,Errors
date,All valid,-
target,All valid,-



Running search request, search_id=f9b437f4-91e1-4b1f-9e25-ddf4312fa107
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
-

39 relevant feature(s) found with the search keys: ['date']


f_autofe_div_0a5adf97,1.1183,100.0000,"0.1574, 0.1517, -0.3505",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_536f8702,1.1164,100.0000,"0.8846, -0.6879, -0.4568",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_efd66bbe,1.0686,100.0000,"0.6131, 0.1437, 0.927",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_1edd5009,1.0472,100.0000,"0.005, 0.0419, 0.0449",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,0.9956,100.0000,"15.1954, 6.5588, 5.1134",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_cbf285e0,0.7026,100.0000,"0.4262, -0.8299, -1.099",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_25296268,0.6455,100.0000,"0.611, 0.7963, -0.818",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_04d9bc60,0.6416,100.0000,"1.0582, -1.0551, 1.0179",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_b49258bb,0.5871,100.0000,"294.3986, -283.4764, 301.9199",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_b20799dd,0.5855,100.0000,"1.0353, 0.9198, 0.9574",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_5388a2c0,0.4933,100.0000,"1.4925, 0.7926, 1.2263",Upgini,AutoFE: features from Markets data,Daily


Upgini,"AutoFE: features from Calendar data,Markets data",11.3018,23
Upgini,AutoFE: features from Calendar data,1.4208,8
Upgini,AutoFE: features from Markets data,1.3385,3
Upgini,Calendar data,0.0092,4
Upgini,World economic indicators,0.0019,1


"Calendar data,Markets data",f_autofe_mul_efd66bbe,f_events_date_year_cos1_9014a856,f_financial_date_vix_7d_to_1y_634c77eb,*
"Calendar data,Markets data",f_autofe_div_1edd5009,f_events_date_year_cos1_9014a856,f_financial_date_vix_6c8e6194,/
"Calendar data,Markets data",f_autofe_div_0a5adf97,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_92dac942,/
"Calendar data,Markets data",f_autofe_div_cbf285e0,f_events_date_year_cos1_9014a856,f_financial_date_gold_7d_to_7d_1y_shift_a8127a43,/
"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_536f8702,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_1y_7a6f300d,/
"Calendar data,Markets data",f_autofe_div_0c0eef24,f_events_date_week_cos3_7525fe31,f_financial_date_gold_7d_to_7d_1y_shift_a8127a43,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_mul_89f64ca9,f_events_date_week_sin1_847b5db1,f_financial_date_natural_gas_7d_to_1y_7a6f300d,*
Markets data,f_autofe_div_0dcebab1,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_snp500_7d_to_1y_378b9f99,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/


We detected 64 outliers in your sample.
Examples of outliers with maximum value of target:
45    205
25    196
60    180
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,9578,50.2328,306.9850,233.1561,73.8289
Eval 1,2422,59.6908,501.9799,423.4478,78.5321


In [31]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)